In [25]:
using LinearAlgebra, BenchmarkTools

# 行列演算の順番による計算速度の違い

M = 20000, K = 500, N = 10000 の設定で、M×K行列X、K×N行列Y、N次元ベクトルz を用意します。

In [14]:
M = 20000
K = 500
N = 10000

X = randn(M, K)
Y = randn(K, N)
z = randn(N, 1)

10000×1 Array{Float64,2}:
 -0.21379877037268807   
  0.40301162303937016   
 -0.00018076067621689294
  0.2116150976232541    
 -0.6801314941031131    
  0.7322601358177026    
  2.1978663881312777    
  0.08212411992094877   
 -0.9449659702886307    
 -2.698372623528385     
  0.8417969714245113    
  1.1754241499794846    
 -0.4856735800036543    
  ⋮                     
 -0.027144473455445222  
  0.653287683616669     
  0.23656347593079013   
 -0.09544757748084728   
  0.3647486225944841    
 -0.7203140816695207    
  0.2545504278092107    
  1.0999969554458848    
  0.5998074510974148    
  0.07641789194714461   
 -0.007841024578390478  
  0.49979664069719804   

行列演算の順番によって、計算時間は変わります。

1. (X Y) z と計算すると O(M×N×K) + O(M×N×1) = O(M×N×K)
2. X (Y z) と計算すると O(N×K×1) + O(M×K×1) = O((M+N)×K)

になるので、2.で計算したほうが単純計算で O((M×N)/(M+N)) くらい速くなりそうです。  
M = 20000, N = 10000 の場合 2×10^8 / 3×10^4 = 6666 倍です。  
**行列が小さくなる方から計算するというのが行列演算の鉄則**になります。

In [15]:
@benchmark (X * Y) * z

BenchmarkTools.Trial: 
  memory estimate:  1.49 GiB
  allocs estimate:  4
  --------------
  minimum time:     2.346 s (0.02% GC)
  median time:      2.602 s (3.54% GC)
  mean time:        2.602 s (3.54% GC)
  maximum time:     2.859 s (6.43% GC)
  --------------
  samples:          2
  evals/sample:     1

In [16]:
@benchmark X * (Y * z)

BenchmarkTools.Trial: 
  memory estimate:  160.39 KiB
  allocs estimate:  3
  --------------
  minimum time:     7.824 ms (0.00% GC)
  median time:      9.591 ms (0.00% GC)
  mean time:        10.696 ms (0.18% GC)
  maximum time:     36.140 ms (0.00% GC)
  --------------
  samples:          467
  evals/sample:     1

**X (Y z)で計算すると、(X Y) z で計算するより 270倍速くなっています。**  
オーダーで見た見たほど差が出ませんでしたが、それでもかなり高速になっています。

In [17]:
@benchmark X * Y * z

BenchmarkTools.Trial: 
  memory estimate:  1.49 GiB
  allocs estimate:  4
  --------------
  minimum time:     2.643 s (0.02% GC)
  median time:      2.690 s (3.43% GC)
  mean time:        2.690 s (3.43% GC)
  maximum time:     2.738 s (6.73% GC)
  --------------
  samples:          2
  evals/sample:     1

カッコで計算の順番を指定しないと前者の計算時間に近くなってしまっています。

計算結果は当然一致します。

In [18]:
maximum(abs.((X * Y) * z - X * (Y * z)))

7.275957614183426e-12

(X Y) z と X (Y z)　の計算結果の要素ごとの差を取り、絶対誤差の最大値が 7.27E-12 になっていることがわかります。

# 逆行列を明示的に求めない方速いか

In [31]:
M = 10000
A = randn(M, M)
b = randn(M, 1)

10000×1 Array{Float64,2}:
  0.5359597203852016 
  0.5478098198685091 
  0.7130877510342531 
  0.3885252754290513 
 -2.625784568193896  
 -0.6391712144488282 
 -0.38786338292298056
  1.7958994799574337 
  1.1339605518095066 
 -1.176639090056705  
 -1.6796618024298275 
 -0.3362137677302859 
 -0.737670548205233  
  ⋮                  
 -0.6297099958227115 
  0.30344330110839307
 -0.9483808518385909 
 -0.906478560364976  
 -1.5934405308770236 
 -0.8829058748388322 
 -1.1092049149967242 
  1.4580089567340841 
 -0.3428686812179732 
 -0.691230863117705  
 -0.30403663909596407
 -0.5405674115592802 

**逆行列を明示的に求める場合**

In [32]:
@benchmark inv(A) * b

BenchmarkTools.Trial: 
  memory estimate:  767.98 MiB
  allocs estimate:  9
  --------------
  minimum time:     26.951 s (0.00% GC)
  median time:      26.951 s (0.00% GC)
  mean time:        26.951 s (0.00% GC)
  maximum time:     26.951 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

**逆行列を明示的に求めない場合**

In [33]:
@benchmark A \ b

BenchmarkTools.Trial: 
  memory estimate:  763.09 MiB
  allocs estimate:  7
  --------------
  minimum time:     8.181 s (0.01% GC)
  median time:      8.181 s (0.01% GC)
  mean time:        8.181 s (0.01% GC)
  maximum time:     8.181 s (0.01% GC)
  --------------
  samples:          1
  evals/sample:     1

**inv(A) を明示的に求めず、 A x = b を直接解く A \ b の方が3倍以上高速になっています。**  
繰り返し inv(A) を用いる場合や、事後分布の共分散行列を求める場合など inv(A) の値自体が必要な場合以外は、inv(A) は明示的に求めない方が良いです。

計算結果は当然一致します。

In [34]:
maximum(abs.(inv(A) * b - A \ b))

4.5428105721612155e-12